In [1]:
import time
import requests
import pandas as pd
from datetime import datetime

# Your Stack Exchange API key
api_key = "rl_1FihbmqnnC3CSntbnnbVRkvtC"  # Replace with your actual API key

# Define the number of records and pagesize
total_records = 10000
pagesize = 100  # Max allowed pagesize by the API

def fetch_user_data():
    all_data = []
    page = 1
    while len(all_data) < total_records:
        url = (
            f"https://api.stackexchange.com/2.3/users"
            f"?order=desc&sort=creation&site=stackoverflow"
            f"&pagesize={pagesize}&page={page}&key={api_key}"
        )
        response = requests.get(url)
        if response.status_code == 200:
            results = response.json().get('items', [])
            if not results:
                # No more results
                break
            for user in results:
                data_row = {
                    'account_id': user.get('account_id'),
                    'reputation': user.get('reputation'),
                    'user_id': user.get('user_id'),
                    'user_type': user.get('user_type'),
                    'profile_image': user.get('profile_image'),
                    'display_name': user.get('display_name'),
                    'link': user.get('link'),
                    'creation_date': datetime.fromtimestamp(user.get('creation_date')),
                    'last_access_date': datetime.fromtimestamp(user.get('last_access_date')),
                }
                all_data.append(data_row)
                if len(all_data) >= total_records:
                    break
            page += 1
        else:
            print(f"Error fetching data: {response.status_code}")
            break
        # Add time.sleep to respect API rate limits
        time.sleep(1.1)  # Sleep for 1.1 seconds
    return all_data

# Fetch data
print("Fetching user data from Stack Exchange API...")
data = fetch_user_data()
print(f"Fetched {len(data)} records.")

# Create pandas DataFrame from data
df = pd.DataFrame(data)

# Add index column
df.reset_index(inplace=True)
df.rename(columns={'index': 'id'}, inplace=True)

# Reorder columns if needed
desired_columns = [
    'id', 'account_id', 'reputation', 'user_id', 'user_type',
    'profile_image', 'display_name', 'link', 'creation_date',
    'last_access_date'
]
df = df[desired_columns]

# Save DataFrame to CSV
output_file = 'latest_stack_overflow_user_data.csv'
df.to_csv(output_file, index=False)

print(f'Data saved to {output_file}')


Fetching user data from Stack Exchange API...
Error fetching data: 400
Fetched 100 records.
Data saved to latest_stack_overflow_user_data.csv
